In [31]:
import pandas as pd
import numpy as np

In [3]:
results_session = pd.read_csv("./da_session.csv")
results_subject = pd.read_csv("./da_subject.csv")

### Cross Session

In [4]:
results_session

,Unnamed: 0,align,no_align,time,distance,n_proj,n_epochs,seed,subject,target_subject,cross_subject,multifreq
0,0,0.797153,0.775801,15.365843,lew,500,500,2473,1,0,False,False
1,1,0.797153,0.775801,15.365843,lew,500,500,2473,1,0,False,False
2,2,0.797153,0.775801,15.365843,lew,500,500,2473,1,0,False,False
3,3,0.797153,0.775801,15.365843,lew,500,500,2473,1,0,False,False
4,4,0.797153,0.775801,15.365843,lew,500,500,2473,1,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
370,370,0.750000,0.768939,8.596683,spdsw,500,500,929,9,0,False,False
371,371,0.750000,0.768939,8.596683,spdsw,500,500,929,9,0,False,False
372,372,0.750000,0.768939,8.596683,spdsw,500,500,929,9,0,False,False
373,373,0.750000,0.768939,8.596683,spdsw,500,500,929,9,0,False,False


In [9]:
distances = results_session["distance"].unique()
subjects = results_session["subject"].unique()

In [43]:
L_session = {}
for distance in distances:
    L_session[distance] = {}
L_session["no_align"] = {}


for distance in distances:
    for s in subjects:
        dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}

        scores = results_session[
            (results_session["distance"] == distance)
            & (results_session["subject"] == s)
        ]["align"]
        m_score = scores.mean()
        std_score = scores.std()
        
        times = results_session[
            (results_session["distance"] == distance)
            & (results_session["subject"] == s)
        ]["time"]
        m_t = times.mean()
        std_t = times.std()
        
        dico_distances["mean_score"] = m_score
        dico_distances["std_score"] = std_score
        dico_distances["mean_time"] = m_t
        dico_distances["std_time"] = std_t
        
        L_session[distance][s] = dico_distances
        
        
for s in subjects:
    dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}

    scores = results_session[
        (results_session["distance"] == distance)
        & (results_session["subject"] == s)
    ]["no_align"]
    m_score = scores.mean()
    std_score = scores.std()

    dico_distances["mean_score"] = m_score
    dico_distances["std_score"] = std_score

    L_session["no_align"][s] = dico_distances

In [44]:
L_session

{'lew': {1: {'mean_score': 0.797153024911032,
   'std_score': 0.0,
   'mean_time': 14.57733449935913,
   'std_time': 0.4087868335345795},
  3: {'mean_score': 0.8424908424908425,
   'std_score': 0.0,
   'mean_time': 14.257566928863525,
   'std_time': 0.07702371871575027},
  7: {'mean_score': 0.7732851985559567,
   'std_score': 0.0014738205431908621,
   'mean_time': 14.611153984069825,
   'std_time': 0.07044683176896759},
  8: {'mean_score': 0.8302583025830258,
   'std_score': 0.0,
   'mean_time': 14.183421277999878,
   'std_time': 0.033073793871795834},
  9: {'mean_score': 0.7492424242424243,
   'std_score': 0.0028930402114620175,
   'mean_time': 13.051070404052734,
   'std_time': 0.051286677483824324}},
 'les': {1: {'mean_score': 0.797153024911032,
   'std_score': 0.0,
   'mean_time': 19.882094144821167,
   'std_time': 0.8658042366445823},
  3: {'mean_score': 0.8424908424908425,
   'std_score': 0.0,
   'mean_time': 19.54542202949524,
   'std_time': 1.1287816102523742},
  7: {'mean_scor

In [54]:
acc_session = np.zeros((len(subjects), len(distances)+1))
t_session = np.zeros((len(subjects), len(distances)))

for i, distance in enumerate(distances):
    for j, s in enumerate(subjects):
        acc_session[j, i] = L_session[distance][s]["mean_score"]
        t_session[j, i] = L_session[distance][s]["mean_time"]
        
distance = "no_align"        
for j, s in enumerate(subjects):
    acc_session[j, -1] = L_session[distance][s]["mean_score"]

In [56]:
pd.DataFrame(acc_session, index=subjects, columns=np.concatenate([distances, ["no_align"]]))

,lew,les,spdsw,no_align
1,0.797153,0.797153,0.795018,0.775801
3,0.842491,0.842491,0.849084,0.765568
7,0.773285,0.768953,0.745848,0.690253
8,0.830258,0.830258,0.828044,0.760148
9,0.749242,0.752273,0.749242,0.767424


In [57]:
pd.DataFrame(t_session, index=subjects, columns=distances)

,lew,les,spdsw
1,14.577334,19.882094,8.915159
3,14.257567,19.545422,8.868392
7,14.611154,15.969017,8.869594
8,14.183421,20.467210,8.835149
9,13.051070,30.688083,8.584994


### Cross Subjects

In [5]:
results_subject

,Unnamed: 0,align,no_align,time,distance,n_proj,n_epochs,seed,subject,target_subject,cross_subject,multifreq
0,0,1.000000,1.000000,0.000000,lew,500,500,2473,1,1,True,False
1,1,0.633333,0.507407,15.164546,lew,500,500,2473,1,3,True,False
2,2,0.483395,0.420664,14.447224,lew,500,500,2473,1,7,True,False
3,3,0.662879,0.356061,14.919306,lew,500,500,2473,1,8,True,False
4,4,0.464135,0.261603,14.286490,lew,500,500,2473,1,9,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
370,370,0.410256,0.252747,8.614837,spdsw,500,500,929,9,1,True,False
371,371,0.385185,0.307407,8.602805,spdsw,500,500,929,9,3,True,False
372,372,0.210332,0.247232,8.605646,spdsw,500,500,929,9,7,True,False
373,373,0.488636,0.375000,8.632351,spdsw,500,500,929,9,8,True,False


In [58]:
distances = results_subject["distance"].unique()
subjects_src = results_subject["subject"].unique()
subjects_tgt = results_subject["target_subject"].unique()

In [70]:
L_subject = {}
for distance in distances:
    L_subject[distance] = {}
L_subject["no_align"] = {}


for distance in distances:
    for s1 in subjects_src:
        L_subject[distance][s1] = {}
        for s2 in subjects_tgt:
            if s1 != s2:
                L_subject[distance][s1][s2] = {}
        
                scores = results_subject[
                    (results_subject["distance"] == distance)
                    & (results_subject["subject"] == s1)
                    & (results_subject["target_subject"] == s2)
                ]["align"]
                m_score = scores.mean()
                std_score = scores.std()
                
                times = results_subject[
                    (results_subject["distance"] == distance)
                    & (results_subject["subject"] == s1)
                    & (results_subject["target_subject"] == s2)
                ]["time"]
                
                m_t = times.mean()
                std_t = times.std()
                
                L_subject[distance][s1][s2]["mean_score"] = m_score
                L_subject[distance][s1][s2]["std_score"] = std_score
                L_subject[distance][s1][s2]["mean_time"] = m_t
                L_subject[distance][s1][s2]["std_t"] = std_t
        
        
for s1 in subjects_src:
    L_subject["no_align"][s1] = {}
    for s2 in subjects_tgt:
        if s1 != s2:
            dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}

            scores = results_subject[
                (results_subject["distance"] == distance)
                & (results_subject["subject"] == s1)
                & (results_subject["target_subject"] == s2)
            ]["no_align"]
            m_score = scores.mean()
            std_score = scores.std()

            dico_distances["mean_score"] = m_score
            dico_distances["std_score"] = std_score

            L_subject["no_align"][s1][s2] = dico_distances

In [71]:
L_subject

{'lew': {1: {3: {'mean_score': 0.6333333333333333,
    'std_score': 0.0,
    'mean_time': 14.506327533721924,
    'std_t': 0.37020013167089444},
   7: {'mean_score': 0.4833948339483395,
    'std_score': 0.0,
    'mean_time': 14.345209312438964,
    'std_t': 0.08878611777268253},
   8: {'mean_score': 0.6628787878787878,
    'std_score': 0.0,
    'mean_time': 14.894848394393922,
    'std_t': 0.07626159472480884},
   9: {'mean_score': 0.4641350210970464,
    'std_score': 0.0,
    'mean_time': 14.17175259590149,
    'std_t': 0.10118518932353752}},
  3: {1: {'mean_score': 0.6153846153846154,
    'std_score': 0.0,
    'mean_time': 14.4496515750885,
    'std_t': 0.030167045714461566},
   7: {'mean_score': 0.5756457564575646,
    'std_score': 0.0,
    'mean_time': 14.325219535827637,
    'std_t': 0.0213595518136941},
   8: {'mean_score': 0.6856060606060606,
    'std_score': 0.0,
    'mean_time': 14.729576301574706,
    'std_t': 0.08159189196210861},
   9: {'mean_score': 0.569620253164557,
    

In [90]:
results_mean = np.zeros((len(subjects_src), len(distances)+1))
t_subjects = np.zeros((len(subjects_src), len(distances)))

for l, distance in enumerate(L_subject):
    results = np.zeros((5,5))
    ts = np.zeros((5,5))
    for i, s1 in enumerate(subjects_src):
        for j, s2 in enumerate(subjects_tgt):
            if s1 != s2:
                results[i, j] = L_subject[distance][s1][s2]["mean_score"]
                if distance != "no_align":
                    ts[i, j] = L_subject[distance][s1][s2]["mean_time"]
                
        results_mean[i, l] = np.sum(results[i, :])/4
        
        if distance != "no_align":
            t_subjects[i, l] = np.sum(ts[i, :])/4
                       
    results_pd = pd.DataFrame(results, index=subjects_src, columns=subjects_tgt)
    print(distance)
    print(results_pd)
    print()

lew
          1         3         7         8         9
1  0.000000  0.633333  0.483395  0.662879  0.464135
3  0.615385  0.000000  0.575646  0.685606  0.569620
7  0.418315  0.507407  0.000000  0.420455  0.459916
8  0.593407  0.718519  0.435424  0.000000  0.540084
9  0.429304  0.449630  0.336531  0.542424  0.000000

les
          1         3         7         8         9
1  0.000000  0.633333  0.476015  0.666667  0.464135
3  0.615385  0.000000  0.579336  0.685606  0.565401
7  0.417582  0.511111  0.000000  0.420455  0.468354
8  0.593407  0.722222  0.435424  0.000000  0.540084
9  0.435897  0.442222  0.346863  0.548485  0.000000

spdsw
          1         3         7         8         9
1  0.000000  0.583704  0.458303  0.625758  0.486920
3  0.604396  0.000000  0.502583  0.702273  0.527426
7  0.444689  0.505926  0.000000  0.376515  0.438819
8  0.575824  0.714074  0.417712  0.000000  0.507173
9  0.388278  0.417037  0.256827  0.481061  0.000000

no_align
          1         3         7       

In [83]:
pd.DataFrame(results_mean, index=subjects_src, columns=np.concatenate([distances, ["no_align"]]))

,lew,les,spdsw,no_align
1,0.560935,0.560037,0.538671,0.386434
3,0.611564,0.611432,0.584169,0.348530
7,0.451523,0.454376,0.441487,0.339370
8,0.571858,0.572784,0.553696,0.400085
9,0.439472,0.443367,0.385801,0.297457


In [91]:
pd.DataFrame(t_subjects, index=subjects_src, columns=distances)

,lew,les,spdsw
1,14.479534,17.840664,8.844894
3,14.424035,18.544150,8.816453
7,14.507743,16.931360,8.833624
8,14.528703,17.177580,8.784677
9,14.007748,19.393099,8.606318
